{[Click aquí para ver este documento en Google Colab](https://colab.research.google.com/drive/1cU9MHxxTRqwwqdGPfM6COcik6lZD017l)}

<head><link rel = "stylesheet" href = "https://drive.google.com/uc?id=1zYOH-_Mb9jOjRbQmghdhsmZ2g6xAwakk"></head>

<table class = "header" width = 100%><tr>
    <th align = "left">The S<code>AI</code>X Guys | AI Trading, Octubre 2020</th>
    <th align = "right">Sección: ZMQ</th>
</tr></table>

## <center><b><u>Clase "MTrack"</u></b></center>

Nuestro interactor con la fuente de datos de mercado va a ser esta clase. Fundamentalmente, se encargará de 3 cosas:
* El manejo de una base de datos "``Data``": cada instrumento y cada marco temporal específico, con su DataFrame OHLCVS.
* La suscripción a datos de mercado en tiempo real, en cualquier marco temporal (ya sea regular o irregular).
* La descarga de datos de mercado históricos en CSV y el posterior importado y enlazado con el resto de la base de datos.

Basandonos en la ya creada superclase "``ZMQL``" y en la implementación en [ZeroMQ de Darwinex](https://www.darwinex.com/es/algorithmic-trading/zeromq-metatrader), se tendrán 3 canales de comunicación entre "``MTrack``" y MetaTrader. Por ende, el bloque "``MTrack``" va a necesitar 3 sockets fundamentales:
* "**PUSH**", para enviar solicitudes y comandos hacia (el PULL de) MetaTrader.
* "**PULL**", para recibir respuestas y confirmaciónes desde (el PUSH de) MetaTrader.
* "**SUB**", para recibir el flujo continuo y asincrónico de datos desde (el PUB de) Metatrader.

El esquema básico vendría a ser el siguiente.
<center><img width = "75%" src = "https://drive.google.com/uc?id=1vqcI96ZYoR5AdZGDONrtLxtSSlPQtAd8"></img></center>

En las proximas secciones iremos explicando cada una de las funciones y objetos que conforman al bloque. Los módulos que vamos a usar de Python ademas de las de ZMQ, son:
* "``datetime``": Funciones de calendario. Especialmente "``datetime.now``" y las funciones de "``timedelta``" y "``timestamp``".
* "``pandas``": Lógicamente, todo lo relacionado con DataFrames para manipulación de datos de tick, y velas OHLCVS.
* "``IPython``": De nuevo, para poder visualizar DataFrames como tablas en HTML, y evitar verlas como "``monospace``".

En el futuro programa principal, se deberá crear un único "``Context``" para todos los nodos de la red. Pero como el Kernel de este notebook solamente se encargará de estudiar la clase "``MTrack``" (que es en sí un objeto ``ZMQL``), vamos a crear un "``Context``" ahora independientemente de si hay otros notebooks funcionando en paralelo a este.

In [1]:
import datetime, pandas, time, ZMQL, sys
from IPython.display import display
from zmq import Context
ZMQ = Context()

### <center><u><b>Preparativos</b></u></center>

A esta sección la vamos a ocupar en crear algunas primeras herramientas de proposito general. Por sobre todo, porque resumerían fragmentos de código que aparecen repetidas veces, mas adelante. Ciertas de ellas son funciones que además pueden resultar útiles también fuera del objeto, ya sea para uso particular, o para debugging.

<b><u>Constantes de ZMQL</u></b>: Formalmente llamados "atributos de clase". Habíamos guardado a algunos dentro de ``ZMQL``, ya que podrían llegar a necesitarse dentro de cualquier futuro bloque. Por ejemplo, "``_MQErrors``" que contiene el listado de [errores numerados de MQL4](https://docs.mql4.com/constants/errorswarnings/errorcodes). A todos ellos los nombraremos con un primer guión bajo "``_``", y algunas letras mayúsculas delante para diferenciarlas de los métodos internos como "``_check``". Tenemos:
* "``_SymError``", "``_TFError``" y otras futuras descripciones o avisos de casos de errores que pudieran llegar a ser comúnes a mas de una función.
* "``_TFLabels``", con las letras que representan cada unidad de tiempo dentro de la designación estandar de los marcos temporales en MetaTrader.

<b><u>Función "``_check_symbol``"</u></b>: tener en cuenta que una excepción "grave" como "``AssertionError``" podría llegar a hacer que Python detenga todo el sistema de trading de manera automática. Algunos "``asserts``" pueden resultar de simples equivocaciones. Por ejemplo, querer subscribirse a EURUSD o cualquier otro simbolo y escribirlo mal.

Básicamente, "``_check_symbol``" toma al símbolo escrito como ``str``ing, y si salió todo bien, lo devuelve en mayuscula (de modo que podemos ingresarlo en minúscula si queremos). Caso contrario, hace un ``print`` del error **sin arrojar una excepción**, y devuelve un "``None``" que puede despues ser utilizado en cualquier función para reaccionar debidamente.

<b><u>Función "``_check_frame``"</u></b>: MetaTrader clasifica a las temporalidades con estandares (``str``ings) como "``M1``", "``H4``", etc. Para ahorrar código a futuro, es conveniente tener una función como esta que simplemente tome dichos ``str``ings y los convierta en un ``tuple`` como "``("minutes", 1)``" o "``("hours", 4)``". Al mismo tiempo, si hay un error, actua similar a "``_check_symbol``".

In [2]:
class MTrack(ZMQL.ZMQL):
    _TFLabels = {"T": "ticks", "Z": "milliseconds", "S": "seconds", "M": "minutes", "H": "hours", "D": "days"}
    _SymError = "\"symbol\" must be given as a string and must be enlisted in Broker's MetaTrader's symbol watch!"
    _TFError = "\"frame\" must carry one of the MetaTrader's standardized timeframes! (e.g.: \"M5\", \"H4\", etc.)"
    def __init__(self, context, verbose = 1):
        ports = {"SUB": 65530, "PUSH": 65531, "PULL": 65532}
        super().__init__(context, ID = "MTrack", ports = ports, verbose = verbose)
    @staticmethod
    def _check_symbol(symbol, action = None):
        if isinstance(symbol, str): return symbol.upper()
        print(f"(({action})) ERROR! {MTrack._SymError}")
        return None
    @staticmethod
    def _check_frame(frame, action = None):
        try: #| La separación solo funciona si "frame" es string.
            time_unit, N = frame[0], int(frame[1:]) #| Separar en letra y número, sabiendo...
            time_unit = MTrack._TFLabels[time_unit] #| ...que la letra siempre viene primero.
            return time_unit, N
        except: #| En caso de error, reportar y devolver 2 Nones.
            print(f"(({action})) ERROR! {MTrack._TFError}")
            return None, None #| Un None por letra, el otro por número.

#### <b><u>Marcos temporales</u></b>

Si bien el etiquetado de marcos temporales viene de MetaTrader, internamente MQL no reconoce estos ``str``ings. Él los entiende como un número igual a la cantidad de minutos entre vela y vela. De hecho, MQL cuenta con una ``enum``eración con dichos valores ya definidos. Ver la tabla inferior para entender esto mejor.

<u><b>Función "``_frame_enum``"</b></u>: Otras temporalidades no son reconocibles por MQL4; al menos no de manera directa. O sea: MetaTrader no guarda datos OHLCV de "``M10``", o de "``H12``", o de una temporalidad menor como "``S10``". Pero uno si lo desea, puede construir las velas por cuenta propia. Y de hecho, eso es a lo que apuntamos. Empezamos por crear nuestra propia ``enum``eración. Será similar a la de MQL, con una diferencia fundamental: En lugar de contar minutos, los "``enum``" **positivos** serán contabilizados en **segundos**:

| &emsp; <u>``frame``</u> &emsp; | &emsp; <u>``time_unit``</u> &emsp; | &emsp; <u>``N``</u> &emsp; | &emsp; <u>``enum``, MQL</u> &emsp; | &emsp; <u>``enum``, PY</u> &emsp; |
|:----------:	|:-------------:	|:-----:	|:-------------:	|:------------:	|
|  ``"M1"`` 	| ``"minutes"`` 	|   1   	|       1       	|      60      	|
|  ``"M5"`` 	| ``"minutes"`` 	|   5   	|       5       	|      300     	|
| ``"M15"`` 	| ``"minutes"`` 	|   15  	|       15      	|      900     	|
| ``"M30"`` 	| ``"minutes"`` 	|   30  	|       30      	|     1800     	|
|  ``"H1"`` 	|  ``"hours"``  	|   1   	|       60      	|     3600     	|
|  ``"H4"`` 	|  ``"hours"``  	|   4   	|      240      	|     14400    	|
|  ``"D1"`` 	|   ``"days"``  	|   1   	|      1440     	|     86400    	|

Además, vamos a agregar 3 elementos nuevos:

1) Permitiremos valores de marcos temporales que no se encuentran en MetaTrader.
<br>Ejemplos:
  * "``S1``" $\rightarrow$ "``("seconds", 1)``" $\rightarrow$ "``enum = 1``".
  * "``S12``" $\rightarrow$ "``("seconds", 12)``" $\rightarrow$ "``enum = 12``".
  * "``M20``" $\rightarrow$ "``("minutes", 12)``" $\rightarrow$ "``enum = 1200``".
  * "``H3``" $\rightarrow$ "``("hours", 12)``" $\rightarrow$ "``enum = 10800``".

2) Permitiremos valores de "``enum``" positivos **entre 0 y 1**, con una máxima precisión de 0.1:
<br>Ejemplos:
  * "``Z100``" $\rightarrow$ "``("milliseconds", 100)``" $\rightarrow$ "``enum = 0.1``".
  * "``Z200``" $\rightarrow$ "``("milliseconds", 200)``" $\rightarrow$ "``enum = 0.2``".
  * "``Z500``" $\rightarrow$ "``("milliseconds", 500)``" $\rightarrow$ "``enum = 0.5``".

3) Permitiremos valores de "``enum``" **negativos**: los cuales serán múltiplos de **ticks**.
<br>Ejemplos:
  * "``T1``" $\rightarrow$ "``("ticks", 1)``" $\rightarrow$ "``enum = -1``". Estos serían de hecho, **datos de ticks** en sí.
  * "``T32767``" $\rightarrow$ "``("ticks", 32767)``" $\rightarrow$ "``enum = -32767``". Este es el máximo permitido.

Observar que entonces, las letras ``S``, ``M``, ``H`` y ``D`` al final se terminan comportando como multiplicadores. Primero nos hacemos cargo de que la expresión está bien escrita con "``_check_frame``", y luego construimos el valor de "``enum``" acorde a nuestras reglas.

In [3]:
class MTrack(MTrack):
    def __init__(self, context, verbose = 1):
        super().__init__(context, verbose = verbose)
    @staticmethod
    def _frame_enum(frame):
        time_unit, N = MTrack._check_frame(frame)
        if (frame[0] == "T"): enum = -N
        if (frame[0] == "Z"): enum = N/1000
        if (frame[0] == "S"): enum = N
        if (frame[0] == "M"): enum = N*60
        if (frame[0] == "H"): enum = N*60*60
        if (frame[0] == "D"): enum = N*60*60*24
        return round(enum*10)/10 #| Redondear a 0.1

<u><b>Constante "``_DColumns``"</b></u>: visualmente, ya conocemos el proceso de construcción de una vela de gráfica en función a puntos de precios, u otras velas de menor marco temporal...

<center><img width = "80%" src = "https://drive.google.com/uc?id=10BA-85chwZleAr0BKAGteeXjlXMhjNej"></img></center>

Dentro de un DataFrame lleno de datos, la vela está representada por cada una de las filas. Cada fila contiene su propio valor de "``Open``", "``High``", "``Low``" y "``Close``". Por eso se los llama "datos ``OHLC``". Si además incluye datos de "``Volume``", se los llama "datos ``OHLCV``". Como MetaTrader, nuestro sistema contemplará también los "``Spread``", con lo cual trabajaremos con datos "``OHLCVS``".

Análogamente al proceso gráfico, nuestra tarea va a consistir en en:
* "``Open``": encontrar el primer ("``first``") valor de "``Open``" en todas las filas del intervalo.
* "``High``": encontrar el mas alto ("``max``") valor de "``High``" en todas las filas del intervalo.
* "``Low``": encontrar el mas bajo ("``min``") valor de "``Low``" en todas las filas del intervalo.
* "``Close``": encontrar el último ("``last``") valor de "``Close``" en todas las filas del intervalo.
* Para "``Volume``", es la cantidad de transacciones que ocurrió en una fila puntual. Por lo tanto, para una fila hecha desde un conjunto de filas, el valor representativo de "``Volume``" es la cantidad de transacciones totales ("``sum``").
* Para "``Spread``", hay que tener en cuenta que es una variable que nos juega en contra. Conviene entonces siempre contemplar el caso mas crítico/negativo. Nos quedamos entonces con el ``Spread`` mas alto ("``max``").

<u><b>Función "``_reframe``"</b></u>: Pandas tiene herramientas que son capaces de tomar directamente estas etiquetas, y formular la vela/fila por si sola. Las pondremos en práctica dentro de la función "``_reframe``".

Antes que nada, tener en cuenta que todo este proceso solamente tiene validez si el marco temporal (regular o irregular/ticks) tiene que ser un **divisor entero** al de salida. Eso es porque "no se puede armar media vela", o "20.25 velas". Algunos ejemplos:
* Con 2 velas de "``M1``", se puede hacer 1 vela de "``M2``". Factor: 2.
* Con 5320 velas de "``S5``", se pueden hacer 2 velas de "``H3``". Factor: 2160.
* Con 10000 velas de "``T50``", se pueden hacer 200 velas de "``T2500``". Factor: 50.
* De ningúna manera se pueden hacer velas de "``M1``" con velas de "``M2``". Factor: 1/2.
* De ninguna manera se pueden hacer velas de "``H12``" con velas de "``H10``". Factor: 6/5.

$$ Factor = \frac{marco\;temporal\;final}{marco\;temporal\;inicial} = \frac{Num.\;filas\;inicial}{Num.\;filas\;final} $$

Tampoco se pueden hacer velas de marco temporal regular, desde una irregular ("``T...``"), ya que siempre, los datos provenientes de ticks son justamente la concepción mas representativa posible de la actividad del mercado. Sin embargo, el camino inverso es factible, siempre y cuando los datos de ticks sean lo suficientemente precisos. De hecho, es lo que MetaTrader hace en realidad: en un intervalo de tiempo, compactar tick por tick, en una misma vela. Pero claro: entre tick y tick, en casos extremos, no pasa mas de un par de segundos. Y si el marco temporal mínimo es "``M1``"...

Ahora bien; la función "``_reframe``" requiere de 2 argumentos: la ``data`` OHLCVS a compactar, y el marco temporal al que queremos llevarlos (``frame``).
* Si "``frame``" representa un <u>marco temporal regular</u>: debemos convertirla en un "``datetime.timedelta``", que justamente represente saltos de tiempo interpretables por Pandas. Aprovechamos que los argumentos de dicho objeto son "``minutes``", "``seconds``", etc. Justamente, estos son los "``time_unit``s" que se obtienen como salida de la función "``_check_frame``". Luego, se hace un "``resample``" de nuestra ``data``.
* Si "``frame``" representa un <u>marco temporal irregular, medido en cantidad de ``ticks``</u>: se usa la función "``groupby``". Pero para ello, debemos especificar no cada cuantos ticks (filas) deseamos crear una, sino que <u>cual es el número total</u> de filas que deseamos que queden ("$Num.\;filas\;final$"... o "``rows``" en el código). Por ejemplo: si en mi ``data`` tengo 10000 filas "``T25``" y quiero pasarlas a "``T250``", quedarían un total de "$10000 \times 25/250 =$ 1000 filas".

In [4]:
class MTrack(MTrack):
    _DColumns = {"Open": "first", "High": "max", "Low": "min",
              "Close": "last", "Volume": "sum", "Spread": "max"}
    def __init__(self, context, verbose = 1):
        super().__init__(context, verbose = verbose)
    @staticmethod
    def _reframe(frame, data):
        assert isinstance(data, (pandas.Series, pandas.DataFrame)), \
               "ERROR! \"data\" must be a Pandas' series at least!"
        time_unit, N = MTrack._check_frame(frame) #| Unidad de tiempo, y cantidad de ellas.
        if (time_unit == None): return None #| "frame" inexistente dentro de "_check_frame".
        if (time_unit != "ticks"): #| Marco temporal regular...
            delta = datetime.timedelta(**{time_unit: N}) #| Crear objeto de salto de tiempo.
            data = data.resample(rule = delta)  #| Calcular velas nuevas.
        else: #| Si el marco temporal es irregular...
            rows = N*(numpy.arange(len(data))//N) #| Hallar número de filas final.
            data = data.groupby(data.index[rows]) #| Calcular velas nuevas.
        return(data.agg(MTrack._DColumns)) #| Reformular datos con nuevo "frame".

#### <u><b>Unit test</b></u>

Vamos a poner en práctica estas funciones. Como son estáticas, no necesitamos crear una instancia de "``MTrack``": solo convocarlas desde la clase. Descargaremos datos de [Yahoo! Finance](https://github.com/ranaroussi/yfinance) para probar la capacidad de "``_reframe``" de transformar los DataFrames con diferentes marcos temporales.

<u>Algo importante a tener en cuenta</u>: cuando se usa la función "``resample``" dentro de Pandas con un determinado salto temporal ("``datetime.timedelta``"), ésta tiende a redondear para abajo a las marcas de tiempo, de la misma forma que lo hace MetaTrader:
* Ubica la unidad temporal directamente superior mas cercana. Ej.: segundo $\rightarrow$ minuto, minuto $\rightarrow$ hora, etc.
* Divide a dicha unidad en saltos temporales iguales al marco temporal. Ej.: "M15" $\rightarrow$ HH:00, HH:15, HH:30, HH:45, etc.
* Redondea para abajo a la primera marca de tiempo de ``data`` hacia el salto temporal mas próximo. Ej.: 9:48 PM a "``M15``" $\rightarrow$ 9:45 PM.

Algunos ejemplos de todo esto:
* Quiero pasar ``data`` en "``M15``" comenzando a las 15:15 hacia "``H1``".
<br>Lo esperado sería que las marcas de salida fueran 15:15, 16:15, 17:15, etc.
<br>Pero no: se obtendrán las marcas de 15:00, 16:00, 17:00, etc.
* Quiero pasar ``data`` en "``S10``" comenzando a las 2:17:30, hacia "``M5``".
<br>Lo esperado sería que las marcas de salida fueran 2:17:30, 2:22:30, etc.
<br>Pero no: se obtendrán las marcas de 2:15:00, 2:20:00, 2:25:00, etc.

<u>Esto no sucede con los marcos temporales irregulares</u>. Esto es porque los ticks por definición son eventos reales y únicos, y sus marcas temporales no son simplemente aproximaciones temporales. La marca de tiempo relevante es entonces siempre aquella primera dentro de una serie de velas siendo compactada.

<b><u>Nota</u>:</b> Probablemente no usaremos "``_reframe``" dentro del mecanismo, ya que MQL se encargará del empaquetado de velas personalizadas. Lo hará secuencialmente, a medida que los "ticks" van llegando a MetaTrader. Sin embargo, es imprescindible conservar esta función, ya sea para uso externo, o para reformular bases de datos en el transcurso de la operatoria (aunque esto no se aconseja).

In [5]:
from yfinance import download
from numpy.random import randint
#| Creamos una forma mas completa de "yfinance.download".
def ydownload(symbol, frame, dt1, dt2):
    style = "%Y-%m-%d %H:%M:%S"
    t = datetime.datetime.now() #| Marca de tiempo actual.
    dt1 = datetime.timedelta(**dt1) #| Marca de tiempo DF más reciente.
    dt2 = datetime.timedelta(**dt2) #| Marca de tiempo DF más lejana.
    frame = (frame[1:] + frame[0]).lower()
    df = download(tickers = symbol, end = t - dt2,
                interval = frame, start = t - dt1)
    df.index = pandas.to_datetime(df.index, format = style)
    df.drop(columns = "Adj Close", inplace = True)
    df["Spread"] = randint(1, 10, size = len(df))/10
    return df.round(2)
#| Descarga de datos de Yahoo, y posterior "reframe".
DATA = ydownload("MMM", "M1", {"days": 6}, {"days": 0})
DATA_new = MTrack._reframe(frame = "M6", data = DATA)
print("\nInicial:\n") ; display(DATA.head(15))
print("\nReframe:\n") ; display(DATA_new.head(3))

[*********************100%***********************]  1 of 1 completed

Inicial:



,Open,High,Low,Close,Volume,Spread
Datetime,,,,,,
2020-10-26 09:31:00-04:00,167.68,167.68,167.68,167.68,2250,0.5
2020-10-26 09:32:00-04:00,167.37,167.93,167.36,167.93,94206,0.1
2020-10-26 09:33:00-04:00,167.79,167.84,167.46,167.58,9725,0.1
2020-10-26 09:34:00-04:00,167.49,167.95,167.45,167.95,5528,0.2
2020-10-26 09:35:00-04:00,167.83,168.09,167.68,167.85,7308,0.9
2020-10-26 09:36:00-04:00,167.79,167.85,167.65,167.74,4181,0.1
2020-10-26 09:37:00-04:00,167.78,167.78,167.35,167.35,6137,0.9
2020-10-26 09:38:00-04:00,167.38,167.53,167.35,167.35,5684,0.5
2020-10-26 09:39:00-04:00,167.28,167.31,167.10,167.10,9464,0.2



Reframe:



,Open,High,Low,Close,Volume,Spread
Datetime,,,,,,
2020-10-26 09:30:00-04:00,167.68,168.09,167.36,167.85,119017,0.9
2020-10-26 09:36:00-04:00,167.79,167.85,166.62,166.71,51404,0.9
2020-10-26 09:42:00-04:00,166.74,167.08,166.46,166.48,47860,0.9


### <center><b><u>``Base`` de datos</u></b></center>

Antes que nada: recordar que...
* <u>Instrumento</u>: es todo activo financiero que el broker nos permite comerciar. Ej.: "oro vs. dolar", "acciones de Amazon", "indice S&P500", etc.
* <u>Simbolo ("``symbol``")</u>: es la clásica abreviatura (``str``ing) que el broker usa para referirse a él. Ej.: "``XAUUSD``", "``AMZN``", "``ES US500``" etc.

El esquema de nuestra ``Base`` de datos va a ser de un ``dict`` cuyas etiquetas van a ser justamente los ``symbol``s. Va a contener los DataFrames OHLCVS con los datos históricos de cada uno de ellos. Además, va a incluir una DataFrame con variables de ``_Config``uración para cada ``symbol``. Las columnas serán:
* "``Frame``": el ``str``ing del marco temporal escogido, usando la notación típica de MetaTrader.
* "``Flag``": "avisa" cuando existe una vela/fila nueva para ser leida por la estrategia asociada.
* "``Slot``": Número de registro que ocupa en "``symbol``" registrado dentro del array de MQL4.
<br>De estas dos últimas, nos encargaremos mas adelante.

<u><b>Función "``_setup_symbol``"</b></u>: básicamente se encargará de preparar el "``dict``" de almacenamiento para un "``symbol``" y su marco temporal "``frame``", si este es nuevo a la ``Base`` de datos. Si no lo es, simplemente reemplaza el marco temporal antiguo ("``prev``") por el nuevo, y luego hace el "``_reframe``". Todo esto, claro está, despues de verificar que ambos argumentos de la función no hayan tenido ningún error (con "``_check_symbol``" y "``_check_frame``").

In [6]:
class MTrack(MTrack):
    def __init__(self, context, verbose = 1):
        super().__init__(context, verbose = verbose)
        #| Futura base de datos. "_Config" va a almacenar variables de cada "symbol".
        self.Base = {"_Config": pandas.DataFrame(columns = ["Frame", "Flag", "Slot"])}
    def _setup_symbol(self, symbol, frame, subject = ""):
        #| Verificar que "symbol" y "frame" sean strings y correctos.
        symbol = MTrack._check_symbol(symbol, subject)
        time_unit, N = MTrack._check_frame(frame, subject)
        #| Interrumpir si "symbol" o "frame" incorrectos. Devolver "None" en tal caso.
        if (time_unit == None) or (symbol == None): return None, None
        if symbol in self.Base: #| Si "symbol" ya estaba registrado en "Base"...
            prev = self.Base["_Config"].loc[symbol, "Frame"] #| Adquirir "frame" actual.
            no_reframe_1 = (MTrack._frame_enum(frame) < MTrack._frame_enum(prev))
            no_reframe_2 = (MTrack._frame_enum(frame) == MTrack._frame_enum(prev)) 
            if not (no_reframe_1 or (no_reframe_2 and (subject == "Ticks"))):
                print(f"(({subject})) \"{frame}\" less accurate than former \"{prev}\".",
                "Restart object and database completion if wishing to decrease timestep.")
                return None, None  #| Nueva "frame" de menor precisión o igual: no hacer nada.
            print(f"[[{subject}]] Resampling \"{symbol}\" from \"{prev}\" to \"{frame}\"...")
            self.Base["_Config"].loc[symbol, "Frame"] = frame #| Reemplazar por "frame" nuevo.
            #| Ante un nuevo "frame", hacer el "reframe" de los datos almacenados hasta ahora.
            self.Base[symbol] = MTrack._reframe(frame, self.Base[symbol])
        else: #| Si "symbol" es nuevo, darle un espacio en "Base", y todos los elementos necesarios.
            self.Base[symbol] = pandas.DataFrame(columns = MTrack._DColumns.keys())
            self.Base["_Config"].loc[symbol, :] = frame, False, None #| Configuración base.
        return symbol, frame #| Devolver "symbol" y "frame" (no None) como prueba de que salió todo bien.

#### Unit Test

Para ir de a poco, vamos a adjuntar un "``symbol``" en nuestra primera instancia de "``MTrack``". Esto es para ver un poco sobre la estructura de la ``Base`` de datos. Recordar que luego de usarla, hay que eliminarla con "``shutdown``" para evitar problemas. El argumento "``subject``" es por ahora, solamente una etiqueta para identificar la función dentro de la cual se convoca a "``_setup_symbol``". Mas que nada servirá para identificar el origen de los posibles ``error``es que surjan. Mas adelante veremos los distintos "``subject``" posibles. Ahora usaremos ``"Unit Test"`` en su lugar.

<b><u>Nota</u>:</b> Como estamos usando un Jupyter Notebook, usamos "``thread = False``" dentro de "``_shutdown``".  El unificado de ``Thread``s puede bloquear el programa, como aclarado en el notebook de **ZMQL**.

In [7]:
INST = MTrack(context = ZMQ, verbose = 2)
INST._setup_symbol("EURUSD", "15S", "Unit Test")
print("\nTest symbol 1: Error...\n ==> Base =", INST.Base)
#| Error intencional: "frame" al revés.
INST._setup_symbol("US500", "S15", "Unit Test")
print("\nTest symbol 2: OK...\n ==> Base =", INST.Base)
INST._shutdown(thread = False)  ;  del INST

------------------------------------------------------------------------------
[[INIT]] SUB connecting to port 65530. Await for response...
[[INIT]] PUSH connecting to port 65531. Await for response...
[[INIT]] PULL connecting to port 65532. Await for response...
------------------------------------------------------------------------------
<<PUSH>> Command sent: [Check;;;;;;;;;]
>>SUB<< Received -> {'Check': 'SUB'}
>>PULL<< Received -> {'Check': 'PUSH'}
[[CHECK]] Successfully initialized and connected to MetaTrader! :)
((Unit Test)) ERROR! "frame" must carry one of the MetaTrader's standardized timeframes! (e.g.: "M5", "H4", etc.)

Test symbol 1: Error...
 ==> Base = {'_Config': Empty DataFrame
Columns: [Frame, Flag, Slot]
Index: []}

Test symbol 2: OK...
 ==> Base = {'_Config':       Frame   Flag  Slot
US500   S15  False  None, 'US500': Empty DataFrame
Columns: [Open, High, Low, Close, Volume, Spread]
Index: []}
------------------------------------------------------------------------

### <center><b><u>Procesamiento de mensajes recibidos</u></b></center>

Es el turno de desarrollar la función "``_process``" para poder interpretar los mensajes recibidos despues de ser ``eval``uados dentro de "``_receive``" en ``ZMQL``. Como primera consigna, debemos tener en cuenta que los mensajes ("``message``") leidos...
* ...en ``PULL``; son respuestas a mensajes ``PUSH`` enviados por "``_send``" en momentos particulares.
* ...en ``SUB``; son un flujo continuo e ininterrumpido de velas, a destinar a la ``Base`` de datos.

Sea cual sea, lo que vamos a hacer es aprovechar que se programó especialmente que MQL envie sus mensajes con formato "``dict``". Cada uno tendrá **un único** item (el ``[0]``), cuyo nombre y valor sepamos y llamamos "``subject``" y "``content``" respectivamente:
<center><h3><b><code>message = {'</code><u>subject</u><code>': </code><u>content</u><code>}</code></b></h3></center>

* "<u>``subject``</u>" identifica el <u>asunto</u> del mensaje: En respuesta a qué solicitud previa es que MQL nos lo envió.
* "<u>``content``</u>" identifica el <u>contenido</u> del mensaje: la información sobre tal asunto, que MQL nos quiere hacer conocer.

Pero la pregunta es: **¿<u>Qué asuntos nos importan</u>?**

1. "<u>**``Ticks``**</u>": con la forma "``message = {'Ticks': _______ }``"...
<br>&emsp;&ensp;Suponer que habíamos enviado ("``_send``") la suscripción a un flujo de datos en particular...
<br>&emsp;...necesitamos que primero, ``MQL`` responda al "``PULL``" verificando que el "``symbol``" exista, si el "``frame``" fue posible, etc.
2. "<u>**``OHLCV``**</u>": con la forma "``message = {'OHLCV': _______ }``"...
<br>&emsp;&ensp;Suponer que habíamos enviado ("``_send``") la solicitud de descarga de un dataset histórico...
<br>&emsp;...necesitamos que primero, ``MQL`` responda al "``PULL``" verificando si los datos estaban presentes, el directorio del CSV, etc.
3. <u>**Datos de mercado en tiempo real**</u> desde "``SUB``".
<br>&emsp;&ensp;En la próxima sección nos encargaremos de esto.

<b><u>Función "``_process``</u></b>": lo que va a hacer entonces, es dividir al "``message``" en su respectivo "``subject``" y "``content``". Dependiendo del primer elemento, va derivando el segundo hacia la función que es responsable de interpretar la respuesta. De ellas ("``_response_``...") nos encargaremos en un momento.

In [8]:
class MTrack(MTrack):
    def __init__(self, context, verbose = 1):
        super().__init__(context, verbose = verbose)
    def _process(self, message):
        #| Divido el "dict" en el asunto y el contenido del mensaje.
        subject, content = list(message.items())[0]
        #| Derivo el contenido a cada función responsable, acorde al asunto.
        if (subject == "Ticks"): self._response_Ticks(content)
        if (subject == "OHLCV"): self._response_OHLCV(content)
        symbol = subject #| Si "subject" es algún "symbol", el "content" es una vela.
        if (subject in self.Base.keys()): self._process_candle(symbol, content)

#### <b><u>Procesamiento de datos de mercado ("``SUB``")</u></b>

El formato que adoptaremos para un dato de tiempo real, es el siguiente:
<center><img width = "90%" src = "https://drive.google.com/uc?id=1w9j7P9qe2FTKziA_LzY4ZlaRukoZbQfI"></img></center>

Sobre la función para suscribirse a los datos de un "``symbol``" tiempo real, nos encargaremos en unos momentos. Sin embargo, de seguro involucrará haber hecho un "``_setup_symbol``" previamente. Por ello, el "``symbol``" debe ya sí o sí tener su propia tabla OHLCVS, y su fila dentro del DataFrame de ``_Config``uración. Teniendo en cuenta esto, se garantiza que si el "``subject``" del mensaje es una etiqueta dentro de "``Base``", seguro que el contenido se tratará de un dato actual de mercado.

Trabajar de esta manera conlleva 3 ventajas:
* No es necesario que el asunto de un mensaje como tal, sea "tick" o "data" o etc. Ya se diferencia del resto de los asuntos.
* El mensaje ya de por sí, es extenso. Al poner al "``symbol``" dentro del "``content``" y no como "``subject``", nos ahorramos de un elemento.

<b><u>Función "``_format_time``</u></b>": Esta función simplemente traduce la marca de tiempo en formato unix que MQL generalmente usa, hacia la marca de tiempo en formato "``YYYY-MM-DD HH:MM:SS.xxxxxx``" que todos conocemos. Dependiendo del huso horario, es conveniente usar la función de Pandas, o la de ``datetime``. El tiempo en [formato unix](https://es.wikipedia.org/wiki/Tiempo_Unix) por convención se expresa como [la cantidad de segundos que pasaron desde año nuevo del año 1970](https://time.is/Unix_time_now).

<b><u>Función "``_process_candle``</u></b>": primero, interpreta al "``content``" con la estructura de aquella última figura. Va dividiendolo y agrega los datos al final del susodicho DataFrame a modo de nueva fila, usando "``T``" ya traducida, como número de fila (``index``). Aquello solo lo hace si es un dato nuevo.

<u>Aquí es donde la "<b>``Flag``</b>" cobra relevancia</u>: próximamente, la clase "``MThink``" (pendiente, a desarrollar) que contiene a todas las estrategias actuando y prediciendo el mercado, será alertada por el evento de esta "``Flag``" haciendose "``True``", y la volverá a hacer "``False``". Pero claro: <u>no sin antes tomar los datos mas recientes de la ``Base`` de datos para ejecutar su predicción</u>.

Observar que se agregó un argumento "``max_rows``": la cual será el límite de cantidad de filas dentro del DataFrame. Por defecto, reparte la constante "``_MaxRows``" entre todos los DataFrames de la ``Base``. La última 3 lineas de la función solamente corroboran que no se pase de tal número, eliminando las filas excedentes que quedaron arriba de todo.

In [9]:
class MTrack(MTrack):
    _MaxRows = 1000000 #| Máxima cantidad de filas en todos los DataFrames dentro de Base.
    def __init__(self, context, verbose = 1):
        super().__init__(context, verbose = verbose)
    def _process_candle(self, symbol, content, max_rows = None):
        T = pandas.to_datetime(content[0], unit = "s") #| Traducir unix a fecha/hora.
        data = self.Base[symbol] #| Tomar base de datos del "symbol".
        if not data.empty and (T <= data.index[-1]): return #| Descartar datos viejos.
        data.loc[T, :] = content[1:]  #| Si son datos recientes, adjuntarlos a la Data.
        self.Base["_Config"].loc[symbol, "Flag"] = True #| Notificar a las estrategias en "MThink".
        #| Máxima cantidad de filas POR instrumento.
        if (max_rows == None): max_rows = MTrack._MaxRows/(len(self.Base) - 1)
        #| Borrar excedentes de Data mas antiguos, para no saturar la memoria.
        if (len(data) > max_rows): data = data.iloc[-max_rows:, :]

#### <b><u>Procesamiento de respuestas (PULL)</u></b>

Las siguientes funciones son justamente las "``_response_``..." apenas mencionadas hace algunas celdas. Veamos una por una...

<b><u>Función "``_response_Ticks``"</u></b>: Supongamos que hicimos el siguiente envío...
<center><h3>"<code>_send(</code>'<code>"Ticks"; "EURUSD"; 120; 1;;;;;;</code>'<code>)</code>"</h3></center>

O sea: ***"comenzá a enviarme datos de ``EURUSD`` en ``M2``"***. (Aquel último ``1`` dentro del mensaje es el ``slot``, del cual hablaremos mas adelante)

Luego, el Expert Advisor en MQL genera su respuesta y me la envía a traves de su ``PUSH``. La recibo en mi ``PULL``, por supuesto. Vamos a clasificar dos tipos de respuesta posible:

&emsp;&ensp;**(1)** "``message = {'OHLCV': ``**[**``'EURUSD', ``**``120``]**``}``" $\rightarrow$ ***"se suscribió exitosamente a los datos de ``EURUSD`` en ``M2``"***

Como nuestro "``enum``" de marco temporal es igual a "cantidad de segundos" cuando es un número positivo, el "``enum = 120``" representa a "``M2``".

&emsp;&ensp;**(2)** "``message = {'OHLCV': ``**(**``'EURUSD', ``**``4066``)**``}``"

El mensaje presenta 2 diferencias importantes con el anterior:
* Tener **<u>paréntesis</u> "``()``"** en lugar de corchetes "``[]``" indicarán que <u>MQL encontró un error al intentar procesar la solicitud</u>.
* El **<u>último número</u> ``4066``**, en tal caso no será el número de filas sino el <u>codigo con el cual MQL identifica tal error</u>.

La ventaja de diferenciar entre ``()`` y ``[]`` yace en que Python puede diferenciar entre ``tuple``s y ``list``s respectivamente. Al parsear el mensaje original con la función "``eval``", el "``content``" se convierte en uno de estos tipos de datos por convención. Entonces podemos reconocer un error de un no-error, tan simple como diferenciando entre el tipo de dato, con la función "``isinstance``".

En caso que haya encontrado un error porque "``content``" fue una "``tuple``", lo que hace es tomar el número al final (``[-1]``) y busca la descripción correspondiente para reportarla en pantalla. La busca dentro de "``_MQErrors``", el listado de errores de MQL4 del cual se habló al principio. Como ya visto en el desarrollo de la clase ``ZMQL``, no es un error terminante porque quiere decir que el usuario cometió algún error al mandar una solicitud, y no por eso debería bloquearse el sistema. Por lo tanto, su ``print`` merece un grado de "``verbose = 1``".

In [10]:
class MTrack(MTrack):
    def __init__(self, context, verbose = 1):
        super().__init__(context, verbose = verbose)
    def _response_Ticks(self, content):
        symbol = content[0]
        if isinstance(content, tuple): #| Si hubo algún error por parte de MQL...
            #| Tomar al nº de error al final de "content", y buscar su descripción.
            error = MTrack._MQErrors[content[-1]]
            error = f"\"{symbol}\" -> \"{error}\"."
            #| Mostrar en pantalla, si el grado de "verbose" es 1 o mayor.
            if (self.Enable["verbose"] >= 1): print(f"((Ticks)) Warning! MQL error: {error}")

<b><u>Función "``_response_OHLCV``</u></b>": La primera parte es similar a la función anterior. Despues de reconocer un "``tuple``" dentro del mensaje, interpreta el error, lo muestra en pantalla según ``verbose``, y termina ahí. Sin embargo, en caso que no haya tenido ningún error, supongamos que en principio habíamos enviado elsiguiente mensaje...

<center><h3>"<code>_send(</code>'<code>"OHLCV"; "EURUSD"; 120; 1603000000; 1603150000;;;;;</code>'<code>)</code>"</h3></center>

O sea: <i>"descargame datos de ``EURUSD`` en ``M2``, desde el 18/Oct/2020 a las 2:46:40, hasta el 19/Oct/2020 a las 20:26:40"</i>. ([Fecha/hora unix](https://time.is/Unix_time_converter))

* Luego, suponer que recibimos la siguiente respuesta: $\rightarrow$ "``message = {'OHLCV': ``**[**``'EURUSD', 120, 1603000240, 1603149400, ``**``1243``]**``}``":

Se leería como: "<i>se creó exitosamente un archivo de nombre "``EURUSD 120 1603000240 1603149400.csv``" con ``1243`` filas con datos de ``EURUSD`` en ``S12`` entre el 18/Oct/2020 a las 2:50:40 y el 19/Oct/2020 a las 20:16:40"</i>. Si, <u>las marcas de tiempo no coinciden</u>, no es que hubo algún error: eso es porque quizás no se hallaron todos los datos exactamente entre esas fechas. Lo que hace MetaTrader es siempre localizar el rango directamente inferior, mas cercano.

En principio, tales archivos siempre van a lo que MetaTrader llama "<u>Common Data Folder</u>". En ``ZMQL`` habíamos predefinido una constante de clase llamada "``_CommonPath``" que tenía el directorio de dicho lugar, para siempre ir a buscar las descargas de MetaTrader allí. Vamos a hacer que todas las descargas de datos históricos por medio de esta clase vayan a aterrizar sobre una subcarpeta llamada "**OHLCV**" dentro de la "Common Data Folder". Del mismo modo, por ejemplo, el 3er y último bloque "``MTrade``" que será el responsable de gestionar las operaciones de trading, va a poder descargar listados de operaciones cerradas en su propia subcarpeta "**Closed**".

La segunda parte de "``_response_OHLCV``" se encarga de importar con "``read_csv``" los datos del archivo notificado por la respuesta. Así, los convierte en una DataFrame llamada "``new_data``", identificando primero a la columna de marcas de tiempo como ``ìndex`` ("``index_col = 0``") y dandoles formato de fecha/hora con "``to_datetime``". Luego, lo agregamos al final del DataFrame del "``symbol``" con "``append``".

In [11]:
class MTrack(MTrack):
    def __init__(self, context, verbose = 1):
        super().__init__(context, verbose = verbose)
    def _response_OHLCV(self, content):
        symbol, frame, t1, t2 = content #| Usamos los datos del mensaje para identificar el archivo.
        if isinstance(content, tuple): #| Si llegó a haber un error, el mensaje contendría una "tuple".
            error = MTrack._MQErrors[content[-1]] #| Obtenemos la descripción del error desde el listado.
            error = f"(\"{symbol}, {frame}\") -> \"{error}\"." #| Armamos el aviso del error para mostrar.
            if (self.Enable["verbose"] >= 1): #| Si el grado de verbose es 1 o mayor...
                print("((OHLCV)) Warning! MQL error:", error) #| Reportamos el error en pantalla.
            return #| Terminamos la función acá, ya que no existe ningún CSV de tal "content".
        datapath = MTrack._CommonPath + f"OHLCV\\{symbol} {60*frame} {t1} {t2}.csv" #| Ubicación del CSV.
        new_data = pandas.read_csv(datapath, index_col = 0) #| CSV a DataFrame. "Datetime" pasa a ser index.
        new_data.index = pandas.to_datetime(new_data.index) #| Identificamos las marcas de tiempo como fecha/hora.
        #| Descartar última fila (incompleta). Al resto, llevarlo a nuestra Data.
        self.Base[symbol] = self.Base[symbol].copy().append(new_data)

#### <u><b>Unit Test</b></u>

Ya estamos listos para probar la descarga y el procesamiento de los primeros datos de ``OHLCV`` y de ``Ticks``. Lógicamente, en aquel mísmo orden: es buena práctica dentro del trading, primero hacer un análisis sobre los datos históricos, y luego comenzar a analizar vela por vela, y dato por dato entrante.

Como las velas con "``frame``" menor a ``M1`` no son naturalmente almacenadas dentro de MetaTrader, con tales marcos temporales no vamos a poder descargar OHLCVs directamente desde MetaTrader. Lo que en ese caso se puede hacer es buscar otras fuentes de datos y armar CSVs dentro de la carpeta OHLCV dentro de la "Common Data Folder". Luego, se usan funciones como "``read_csv``" de Pandas para incluir los datos dentro de ``Base``.

Otra opción posible es no descargar ningún dato, y mantener el sistema concentrado solo en recopilar suficientes datos hasta poder aplicar las estrategias seleccionadas. Sin embargo, aquello implica un tiempo de espera.

In [16]:
INST = MTrack(context = ZMQ, verbose = 2)
symbol, frame, t0 = "BTCUSD", "M1", time.time()
enum = MTrack._frame_enum(frame)                        #| Enumeración propia. NO USAR "T"icks.
INST._setup_symbol(symbol, frame, "Unit Test")          #| Reservar espacio para "symbol"    
INST._send("PUSH", f"OHLCV;{symbol};{enum/60};{6}")     #| Para OHLCV, se usa el "enum" de MQL.
print("\nData, OHLCV:\n" + "‾"*11)
print(INST.Base[symbol].index[-4:])                     #| Mostrar últimas 4 marcas de tiempo.
print("\nIncoming candles:\n" + "‾"*16)
INST._send("PUSH", f"Ticks;{symbol};{enum};0")          #| Suscribirse a datos de "symbol".
while (int((time.time() - t0)/enum) < 4):               #| Esperar a 3 velas.
    sys.stdout.flush()  ;  time.sleep(enum)             #| Mostrar velas que van llegando.
print("\nData, updated:\n" + "‾"*13)
print(INST.Base[symbol].index[-4:])                     #| Mostrar últimas 4 nuevas marcas.
INST._send("PUSH", f"Ticks;{symbol};0;0")               #| Desuscribirse a datos de "symbol".
INST._shutdown(thread = False)  ;  del INST             #| Eliminar instancia de "MTrack".

------------------------------------------------------------------------------
[[INIT]] SUB connecting to port 65530. Await for response...
[[INIT]] PUSH connecting to port 65531. Await for response...
[[INIT]] PULL connecting to port 65532. Await for response...
------------------------------------------------------------------------------
<<PUSH>> Command sent: [Check;;;;;;;;;]
>>SUB<< Received -> {'Check': 'SUB'}
>>PULL<< Received -> {'Check': 'PUSH'}
[[CHECK]] Successfully initialized and connected to MetaTrader! :)
<<PUSH>> Command sent: [OHLCV;BTCUSD;1.0;6;;;;;;]
>>PULL<< Received -> {'OHLCV': ['BTCUSD', 1, 1604089560, 1604089860]}

Data, OHLCV:
‾‾‾‾‾‾‾‾‾‾‾
DatetimeIndex(['2020-10-30 20:28:00', '2020-10-30 20:29:00',
               '2020-10-30 20:30:00', '2020-10-30 20:31:00'],
              dtype='datetime64[ns]', freq=None)

Incoming candles:
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾
<<PUSH>> Command sent: [Ticks;BTCUSD;60.0;0;;;;;;]
>>PULL<< Received -> {'Ticks': ['BTCUSD', 60.0]}
>>SUB<< Received -> 

### <center><b><u>Funciones de usuario</u></b></center>

Con todo lo anterior, ya logramos encargarnos al 100% del procesamiento de los mensajes entrantes. Lo que sigue es basicamente la construcción de funciones (de un poco mas alto nivel, a modo de "wrappers") que usen el susodicho "``_send``" para enviar mensajes con solicitudes predefinidas.

<u><b>Función "``_download``</b></u>": responsable del "<u>asunto ``OHLCV``</u>". Toma como argumentos al "``symbol``" del cual queremos datos históricos, al marco temporal "``frame``", y a la cantidad de filas "``rows``". Por las dudas, primero ejecuta "``_setup_symbol``" para ver si los primeros 2 argumentos fueron ingresados correctamente, y para darles espacio dentro de la ``Base`` de datos si es que el "``symbol``" es nuevo. Finalmente, el "``message``" es formulado y enviado a traves del ``PUSH``.

Como los datos son descargados desde el repositorio disponible en MetaTrader, sucede que:

1. Puede que la cantidad de filas final sea menor que "``rows``" si no hay suficientes datos.
2. El marco temporal solo puede ser alguno de los de la tabla de "Marcos Temporales" mas arriba (sección "Preparativos").
3. Se usa la ``enum``eración del MQL4 (minutos). Alcanza con dividir la que organizamos ("``_frame_enum``") por 60 (segundos).

In [13]:
class MTrack(MTrack):
    _DTError = "\"t1\" & \"t2\" must be datetime inputs."
    _TFError2 = "\"frame\" should be chosen from between MT4 standards."
    _TFMT4s = ["M1", "M5", "M15", "M30", "H1", "H4", "D1", "W1", "MN"]
    def __init__(self, context, verbose = 1):
        super().__init__(context, verbose = verbose)
    def download(self, symbol, frame, rows = 10000):
        if not (frame in MTrack._TFMT4s):
            print("((Download)) ERROR!", MTrack._TFError2, MTrack._TFMT4s); return
        if not isinstance(rows, int):
            print("((Download)) Number of \"rows\" must be an \"int\".") ; return
        symbol, frame = self._setup_symbol(symbol, frame, "OHLCV") #| Crear dict en "Base" si el "symbol" no existe.
        if (symbol == None) or (frame == None): return #| Ante algún error al ingresar "symbol" o "frame", abandonar.
        enum = MTrack._frame_enum(frame)/60 #| Enum en MTrack se mide en segundos. Enum en MQL se mide en minutos.
        self._send(label = "PUSH", message = f"OHLCV;{symbol};{enum};{rows};;;;;;") #| Armar y enviar mensaje.

<u><b>Función "``subscribe``</b></u>": Responsable del "<u>asunto ``Ticks``</u>". Toma como argumentos al "``symbol``" del cual queremos datos históricos y al marco temporal "``frame``". Con lo cual debe comenzar con "``_setup_symbol``" del mismo modo que la función anterior. Sin embargo, se agrega aquel nuevo argumento: "``slot``".

El Expert Advisor de MQL no puede enviar datos de demasiados instrumentos al mismo tiempo. Debemos cuidarnos de no forzar los sockets de ZMQ o la memoria RAM. A modo de restricción, se crea un array vacío llamado "``Enable``" dentro el Expert Advisor durante su inicialización. Éste está limitado a no tener mas de una cierta cantidad de elementos, a los cuales llamaremos "``slot``s".

Por ejemplo, un "``Enable``" de 7 "``slot``s", con 3 ya llenos (el ``0``, el ``3`` y el ``5``):

<center><h3>"<code>Enable = [</code> EURUSD <code>;</code> ______ <code>;</code> ______ <code>;</code> AMZN  <code>;</code> ______ <code>;</code> BTCUSD <code>;</code> ______ <code>]</code>"</h3></center>

Al utilizar la función "``subscribe``", tenemos que especificar que "``slot``" queremos ocupar, de los que hay disponibles. Algunas reglas:
* Si elegimos uno por encima del último disponible, MQL nos contestará con un error en "``_response_Ticks``".
* Si elegimos uno ya ocupado, será sobre-escrito por el "``symbol``" y el "``frame``" especificado en "``subscribe``".
* Esto último puede aprovecharse para cambiar el "``frame``" de algun instrumento ya suscripto en "``Enable``".

<u><b>Función "``unsubscribe``</b></u>": Hace exactamente lo contrario a la anterior, dado un determinado "``symbol``" ya presente en el array de "``Enable``".

In [14]:
class MTrack(MTrack):
    _SlotError = "\"slot\" slot must be an integer. Check amount of slots in EA config."
    def __init__(self, context, verbose = 1):
        super().__init__(context, verbose = verbose)
    def subscribe(self, symbol, frame, slot = 0):
        if not isinstance(slot, int) or (slot < 0): #| Verificar que el "slot" sea un int positivo.
            print("((Subscribe)) ERROR!", MTrack._SlotError) ; return
        symbol, frame = self._setup_symbol(symbol, frame, "Ticks") #| Crear dict en "Base" si el "symbol" no existe.
        if (symbol == None) or (frame == None): return #| Ante algún error al ingresar "symbol" o "frame", abandonar.
        for symbol_in in self.Base["_Config"].index: #| Verificar que el "slot" no esté ya ocupado.
            if (self.Base["_Config"].loc[symbol_in, "Slot"] == slot): #| En caso que lo esté por otro "symbol"...
                self.Base["_Config"].loc[symbol_in, "Slot"] = None #| "Reseteo" su status, y prosigo a reemplazarlo.
        self.Base["_Config"].loc[symbol, "Slot"] = slot #| Guardar para monitoreo local y para desuscribir mas tarde.
        enum_Py = MTrack._frame_enum(frame) #| Convertir de "frame" en string a enum con las reglas ya planteadas.
        self._send(label = "PUSH", message = f"Ticks;{symbol};{enum_Py};{slot};;;;;;") #| Armar y enviar mensaje.
    def unsubscribe(self, symbol):
        if (symbol not in self.Base): return #| No se puede desuscribir de un "symbol" al que nunca me suscribí.
        slot = self.Base["_Config"].loc[symbol, "Slot"] #| Recuperar slot para borrar "symbol" en "Enable" (MQL).
        if (slot >= 0): self._send(label = "PUSH", message = "Ticks;"";0;%d;;;;;;" % slot) #| Armar y enviar mensaje.
        self.Base["_Config"].loc[symbol, "Slot"] = None